In [10]:
#초기 코드
import numpy as np

# RoPE에서 사용할 주기적 회전 변환 함수
def apply_rotary_position_embedding(embedding, position, theta):
    """
    임베딩 벡터에 대한 회전 변환을 적용하는 함수
    :param embedding: 각 토큰의 임베딩 벡터 (numpy array)
    :param position: 토큰의 위치 (p 값)
    :param theta: 회전 각도
    :return: 회전 변환된 임베딩 벡터
    """
    embedding_rotated = np.zeros_like(embedding)
    
    cos_theta = np.cos(position * theta)
    sin_theta = np.sin(position * theta)
    
    # 회전 변환 적용: 짝수와 홀수 차원을 쌍으로 묶어 회전
    for i in range(0, len(embedding) - 1, 2):
        embedding_rotated[i] = cos_theta * embedding[i] - sin_theta * embedding[i + 1]
        embedding_rotated[i + 1] = sin_theta * embedding[i] + cos_theta * embedding[i + 1]
    
    # 홀수 차원이 남으면, 그대로 유지
    if len(embedding) % 2 == 1:
        embedding_rotated[-1] = embedding[-1]

    return embedding_rotated

# 예시 토큰 임베딩 벡터 (4개의 토큰, 3차원 임베딩)
embeddings = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9],
                       [10, 11, 12]])

# 임베딩 차원 (임베딩 벡터의 길이)와 토큰 수 정의
embedding_dim = 3
num_tokens = embeddings.shape[0]

# 회전 각도를 설정 (임베딩 차원에 따라 각도를 다르게 설정할 수 있음)
theta = 0.1  # 예시로 작은 각도 설정

# 토큰마다 RoPE 적용
rotated_embeddings = []
for p in range(1, num_tokens + 1):
    rotated_embedding = apply_rotary_position_embedding(embeddings[p-1], p, theta)
    rotated_embeddings.append(rotated_embedding)

# 결과 출력
print("원래 임베딩 벡터:")
print(embeddings)
print("\nRoPE 적용 후 회전된 임베딩 벡터:")
print(np.array(rotated_embeddings))


원래 임베딩 벡터:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

RoPE 적용 후 회전된 임베딩 벡터:
[[ 0  2  3]
 [ 2  5  6]
 [ 4  9  9]
 [ 4 14 12]]


In [23]:
#넘파이로 구현한 RoPE

In [22]:
import numpy as np
import torch
import pytest
from TorchRotrayEmbedding import RotaryEmbedding

def numpy_rotate_half(x):
    print("== NumPy rotate_half ==")
    print("입력 x:", x.shape)
    x = x.reshape(*x.shape[:-1], -1, 2)
    print("reshaped x:", x.shape)
    x1 = x[..., 0]
    x2 = x[..., 1]
    print("x1:", x1.shape)
    print("x2:", x2.shape)
    x = np.stack((-x2, x1), axis=-1)
    print("stacked x:", x.shape)
    x = x.reshape(*x.shape[:-2], -1)
    print("최종 x:", x.shape)
    return x

def numpy_apply_rotary_embedding(freqs, t, start_index=0, scale=1.0):
    print("== NumPy apply_rotary_embedding ==")
    print("입력 t:", t.shape)
    print("freqs:", freqs.shape)
    t_left = t[..., :start_index]
    end_index = start_index + freqs.shape[-1]
    t_middle = t[..., start_index:end_index]
    t_right = t[..., end_index:]
    print("t_left:", t_left.shape)
    print("t_middle:", t_middle.shape)
    print("t_right:", t_right.shape)
    t_transformed = (t_middle * np.cos(freqs) * scale) + (numpy_rotate_half(t_middle) * np.sin(freqs) * scale)
    print("t_transformed:", t_transformed.shape)
    result = np.concatenate((t_left, t_transformed, t_right), axis=-1)
    print("최종 결과:", result.shape)
    return result

class NumpyRotaryEmbedding:
    def __init__(self, dim, theta=10000):
        self.dim = dim
        self.theta = theta
        # PyTorch 구현과 동일하게 freqs 계산
        self.freqs = 1.0 / (theta ** (np.arange(0, dim, 2, dtype=np.float32) / dim))
        print("NumPy freqs:", self.freqs.shape)

    def get_angles(self, seq_len, offset=0):
        print("== NumPy get_angles ==")
        positions = (np.arange(seq_len, dtype=np.float32) + offset)
        print("positions:", positions.shape)
        freqs = np.outer(positions, self.freqs)
        print("freqs before repeat:", freqs.shape)
        freqs = np.repeat(freqs, repeats=2, axis=-1)
        print("freqs after repeat:", freqs.shape)
        return freqs.astype(np.float32)

    def __call__(self, x, offset=0):
        print("== NumPy RotaryEmbedding 호출 ==")
        seq_len = x.shape[1]
        freqs = self.get_angles(seq_len, offset=offset)
        freqs = freqs[None, :, :]  # (1, seq_len, dim)
        print("freqs after adding batch dimension:", freqs.shape)
        return numpy_apply_rotary_embedding(freqs, x)

@pytest.mark.parametrize("batch_size", [4, 8, 12])
@pytest.mark.parametrize("seq_len", [128, 256])
@pytest.mark.parametrize("dim", [64, 128])
def test_rope(batch_size, seq_len, dim):
    # 입력 데이터 생성
    np.random.seed(0)
    torch.manual_seed(0)
    x_np = np.random.randn(batch_size, seq_len, dim).astype(np.float32)
    x_torch = torch.from_numpy(x_np).float()

    print("입력 데이터 x_np:")
    print(x_np)

    # NumPy 기반 RoPE 적용
    numpy_rope = NumpyRotaryEmbedding(dim=dim)
    x_rotated_np = numpy_rope(x_np)
    print("NumPy 결과 x_rotated_np:", x_rotated_np.shape)
    print("NumPy 결과 x_rotated_np:")
    print(x_rotated_np)

    # PyTorch 기반 RoPE 적용
    torch_rope = RotaryEmbedding(dim=dim)
    x_rotated_torch = torch_rope.rotate_queries_or_keys(x_torch)
    x_rotated_torch = x_rotated_torch.detach().numpy()
    print("PyTorch 결과 x_rotated_torch:", x_rotated_torch.shape)
    print("PyTorch 결과 x_rotated_torch:")
    print(x_rotated_torch)

    # 결과 비교
    difference = np.max(np.abs(x_rotated_np - x_rotated_torch))
    print(f"두 구현 간의 최대 차이: {difference}")
    print("NumPy와 PyTorch 구현 결과 차이 벡터:")
    print(x_rotated_np - x_rotated_torch)
    assert np.allclose(x_rotated_np, x_rotated_torch, atol=1e-6), f"NumPy와 PyTorch 구현 결과가 일치하지 않습니다. 최대 차이: {difference}"

    print("NumPy와 PyTorch 구현 결과가 일치합니다.")

if __name__ == "__main__":
    test_rope(1, 128, 32)

입력 데이터 x_np:
[[[ 1.7640524   0.4001572   0.978738   ...  1.4693588   0.15494743
    0.37816253]
  [-0.88778573 -1.9807965  -0.34791216 ... -0.35955316 -0.8131463
   -1.7262826 ]
  [ 0.17742614 -0.40178093 -1.6301984  ...  0.97663903  0.3563664
    0.7065732 ]
  ...
  [ 0.61334914  1.8436999   0.27109098 ... -0.8586684   0.4361871
    1.5714631 ]
  [ 1.0773149   0.8110897  -2.2315376  ... -0.94772434  1.3992299
   -0.22612372]
  [-1.4388542   0.801301   -0.00333145 ...  0.46387276  0.6176608
    2.496417  ]]]
NumPy freqs: (16,)
== NumPy RotaryEmbedding 호출 ==
== NumPy get_angles ==
positions: (128,)
freqs before repeat: (128, 16)
freqs after repeat: (128, 32)
freqs after adding batch dimension: (1, 128, 32)
== NumPy apply_rotary_embedding ==
입력 t: (1, 128, 32)
freqs: (1, 128, 32)
t_left: (1, 128, 0)
t_middle: (1, 128, 32)
t_right: (1, 128, 0)
== NumPy rotate_half ==
입력 x: (1, 128, 32)
reshaped x: (1, 128, 16, 2)
x1: (1, 128, 16)
x2: (1, 128, 16)
stacked x: (1, 128, 16, 2)
최종 x: (1, 128, 